# 解压文件
# 运行先 pip install mne

In [1]:
# !unzip "data/data112870/BCICIV_2a_mat.zip" -d data/

In [2]:
import mne
import scipy.io as scio
import numpy as np

def read_data(data_path,i):
    data_path = data_path
    DATA = scio.loadmat(data_path)
    DATA = DATA["data"][0]
    DATA = DATA[i] #主程序倒叙读6个
    EEG_DATA = DATA["X"][0][0].transpose(1,0)
    EEG_label = DATA['y'][0][0][:,0]

    ch_names = ['Fz','Fp1','Fp2','AF3','AF4','AF7','AF8','C3','POz','Cz','PO3','C4','PO4','PO5','PO6','PO7','PO8','Oz','O1','Pz','P6','P7','EOG-left','EOG-central','EOG-right'] #Fz是对应，其余随便写的不影响
    ch_types = ['eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg',
                'eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg',
                'eeg','eeg','eog','eog','eog']

    info = mne.create_info(ch_names = ch_names,
                            ch_types=ch_types,
                            sfreq=250)
    info.set_montage('standard_1020')
    raw = mne.io.RawArray(EEG_DATA,info)

    n_times = DATA["trial"][0][0][:,0] #时间戳
    event = np.zeros((4,12),int)
    v,b,n,m=0,0,0,0
    for i in range (0,n_times.shape[0]):
        if EEG_label[i]==1:
            event[0,v]=n_times[i]
            v+=1
        if EEG_label[i]==2:
            event[1,b]=n_times[i]
            b+=1
        if EEG_label[i]==3:
            event[2,n]=n_times[i]
            n+=1
        if EEG_label[i]==4:
            event[3,m]=n_times[i]
            m+=1


    events = np.zeros((48,3),int)
    j=0
    for i in range(events.shape[0]):
        if i<12:
            events[i][0]=event[0][j]
            events[i][2]=1
        elif i<24:
            events[i][0]=event[1][j]
            events[i][2]=2
        elif i<36:
            events[i][0]=event[2][j]
            events[i][2]=3
        elif i<48:
            events[i][0]=event[3][j]
            events[i][2]=4
        j+=1
        if j>=12:
            j=0
    events = sorted(events, key = lambda events: events[0])
    event_id = dict(lefthand=1,righthand=2,feet=3,tongue=4)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, ecg=False,
                    exclude='bads')
    epochs = mne.Epochs(raw, events, event_id, tmin=3, tmax=6, proj=True,baseline=(None, None), picks=picks,preload=True)


    data = epochs.get_data()
    # data =np.array(data).reshape((48*22,751))
    label = EEG_label.repeat(22)
    label=label.reshape(48,22)
    return data,label


# Trian Data

In [3]:
train_path = r"data/A01T.mat"
train_data=np.zeros((6,48,22,751),dtype=np.float32)
train_label=np.zeros((6,48,22),dtype=np.int64)
for i in range(6):
    data,label = read_data(train_path,-(i+1))
    train_data[i]=data
    label = np.array(label[:])
    train_label[i]=label
train_data=train_data.reshape((288,22,751))
train_label=train_label.reshape((288,22))

Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline int

# Test Data

In [4]:
test_path = r"data/A01E.mat"
test_data=np.zeros((6,48,22,751),dtype=np.float32)
test_label=np.zeros((6,48,22),dtype=np.int64)
for i in range(6):
    data,label = read_data(test_path,-(i+1))
    test_data[i]=data
    label = np.array(label[:])
    test_label[i]=label
test_data=test_data.reshape((288,22,751))
test_label=test_label.reshape((288,22))

Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline int

In [5]:
import paddle
import paddle.nn.functional as F
import paddle.nn as nn


In [6]:
class CNN_GRU(nn.Layer):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1D(1,32,9,1,padding='same',data_format='NCL')
        self.padding1 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.conv2 = nn.Conv1D(32,32,9,1,padding='same',data_format='NCL')
        self.padding2 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.conv3 = nn.Conv1D(32,32,9,1,padding='same',
        data_format='NCL')
        self.padding3 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.flatten1 = nn.Flatten()
        self.flatten2 = nn.Flatten()
        # self.gru = nn.GRU(1,64,1)
        self.gru = nn.GRU(2976,64,1)
        self.dense1 = nn.Linear(64,64)
        self.dense2 = nn.Linear(64,4)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
    def forward(self,x):
        x=self.conv1(x)
        x=self.relu1(x)
        x=self.padding1(x)
        x=self.conv2(x)
        x=self.relu2(x)
        x=self.padding2(x)
        x=self.conv3(x)
        x=self.relu3(x)
        x=self.padding3(x)
        x=self.flatten1(x)
        # x = x.unsqueeze(-1)
        x = x.unsqueeze(1)
        x,h=self.gru(x)
        x=self.dense1(x)
        x=self.relu4(x)
        x=self.dropout(x)
        x=self.dense2(x)
        x=self.flatten2(x)
        return x

In [ ]:
input_spec = paddle.static.InputSpec(
    shape=(-1,1,751),
    dtype='float32',
    name='x'
)
model = CNN_GRU()

paddle.summary(model,input_spec)
out = model(paddle.randn((1,1,750)))
print(out.shape)

-------------------------------------------------------------------------------
 Layer (type)       Input Shape            Output Shape           Param #    
   Conv1D-1        [[1, 1, 751]]           [1, 32, 751]             320      
    ReLU-1         [[1, 32, 751]]          [1, 32, 751]              0       
  MaxPool1D-1      [[1, 32, 751]]          [1, 32, 375]              0       
   Conv1D-2        [[1, 32, 375]]          [1, 32, 375]            9,248     
    ReLU-2         [[1, 32, 375]]          [1, 32, 375]              0       
  MaxPool1D-2      [[1, 32, 375]]          [1, 32, 187]              0       
   Conv1D-3        [[1, 32, 187]]          [1, 32, 187]            9,248     
    ReLU-3         [[1, 32, 187]]          [1, 32, 187]              0       
  MaxPool1D-3      [[1, 32, 187]]          [1, 32, 93]               0       
   Flatten-1       [[1, 32, 93]]            [1, 2976]                0       
     GRU-1         [[1, 1, 2976]]    [[1, 1, 64], [1, 1, 64]] 

In [ ]:
loss_function = paddle.nn.CrossEntropyLoss()
# lr_scheduler = paddle.optimizer.lr.CosineAnnealingDecay(
#     learning_rate=LEARNING_RATE,
#     T_max=NUM_EPOCHS
# )
optimizer = paddle.optimizer.Adam(
    learning_rate=1e-4,
    parameters=model.parameters()
)

# 8:2 T

In [ ]:
# data = train_data
# label = train_label-1
# # print(data.shape,label.shape)

# from sklearn.preprocessing import StandardScaler #标准化
# from sklearn.preprocessing import MinMaxScaler
# sk = StandardScaler()
# # data = sk.fit_transform(data)

# from sklearn.model_selection import train_test_split
# # data=np.expand_dims(data,1)
# # label=np.expand_dims(label,1)
# train_data,test_data,train_label,test_label = train_test_split(data,label,train_size=0.8)
# print(train_data.shape,train_label.shape,test_data.shape,test_label.shape)


In [ ]:
# train_data=train_data.reshape((5060,751))
# test_data=test_data.reshape((1276,751))
# train_label=train_label.reshape((5060))
# test_label=test_label.reshape((1276))
# from sklearn.preprocessing import StandardScaler #标准化
# from sklearn.preprocessing import MinMaxScaler
# # sk = StandardScaler()
# sk = MinMaxScaler()
# train_data = sk.fit_transform(train_data)
# test_data = sk.fit_transform(test_data)
# train_data=np.expand_dims(train_data,1)
# test_data=np.expand_dims(test_data,1)
# train_label=np.expand_dims(train_label,1)
# test_label=np.expand_dims(test_label,1)

# 8:2 E

In [ ]:
data = test_data
label = test_label-1
print(data.shape,label.shape)

from sklearn.preprocessing import StandardScaler #标准化
from sklearn.preprocessing import MinMaxScaler
sk = StandardScaler()
# data = sk.fit_transform(data)

from sklearn.model_selection import train_test_split
# data=np.expand_dims(data,1)
# label=np.expand_dims(label,1)
train_data,test_data,train_label,test_label = train_test_split(data,label,train_size=0.8)
print(train_data.shape,train_label.shape,test_data.shape,test_label.shape)
train_data=train_data.reshape((5060,751))
test_data=test_data.reshape((1276,751))
train_label=train_label.reshape((5060))
test_label=test_label.reshape((1276))
from sklearn.preprocessing import StandardScaler #标准化
from sklearn.preprocessing import MinMaxScaler
# sk = StandardScaler()
sk = MinMaxScaler()
train_data = sk.fit_transform(train_data)
test_data = sk.fit_transform(test_data)
train_data=np.expand_dims(train_data,1)
test_data=np.expand_dims(test_data,1)
train_label=np.expand_dims(train_label,1)
test_label=np.expand_dims(test_label,1)

(288, 22, 751) (288, 22)
(230, 22, 751) (230, 22) (58, 22, 751) (58, 22)


In [ ]:
NUM_EPOCHS=100
TRAIN_BATCH_SIZE=28

# 五折交叉验证

In [ ]:
import numpy as np
from paddle.io import Dataset


class MIDataset(Dataset):
    def __init__(self, data,label):
        self.data = data
        self.label = label

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        return data, label

    def __len__(self):
        return len(self.label)

In [ ]:
train_dataset = MIDataset(train_data,train_label)
test_dataset = MIDataset(test_data,test_label)
train_loader = paddle.io.DataLoader(
    train_dataset, 
    batch_size=28, 
    shuffle=True,
    drop_last=True)
test_loader = paddle.io.DataLoader(
    test_dataset, 
    batch_size=28,
    shuffle=True
)
model = CNN_GRU()
loss_function = paddle.nn.CrossEntropyLoss()
optimizer = paddle.optimizer.Adam(
learning_rate=1e-4,
parameters=model.parameters())
model_handler = paddle.Model(model)
model_handler.prepare(optimizer=optimizer,
            loss=loss_function,
            metrics=paddle.metric.Accuracy())
model_handler.fit(train_loader,
        test_loader,
        epochs=100,
        batch_size=28,
        verbose=1)
result=(model_handler.evaluate(test_dataset, batch_size=test_data.shape[0]))
result

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/100
step  20/180 [==>...........................] - loss: 1.3685 - acc: 0.2196 - ETA: 2s - 15ms/step

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 180/180 [==============================] - loss: 1.3689 - acc: 0.2867 - 8ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.3724 - acc: 0.2508 - 4ms/step          
Eval samples: 1276
Epoch 2/100
step 180/180 [==============================] - loss: 1.2532 - acc: 0.4012 - 7ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.4486 - acc: 0.2202 - 4ms/step          
Eval samples: 1276
Epoch 3/100
step 180/180 [==============================] - loss: 1.1388 - acc: 0.5438 - 7ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.3594 - acc: 0.3245 - 4ms/step          
Eval samples: 1276
Epoch 4/100
step 180/180 [==============================] - loss: 0.9651 - acc: 0.6917 - 7ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.8689 - acc: 0.2939 - 4ms/step          
Eval samples: 1276
Epoch 5/100
step 180/180 [==============================] - loss: 0.5273